> Multi-lingual ASR w/Transformers

In [1]:
! pip install datasets
! pip install transformers
! pip install huggingface_hub
! pip install torchaudio
! pip install librosa
! pip install jiwer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 451 kB 5.1 MB/s 
     |████████████████████████████████| 212 kB 68.4 MB/s 
     |████████████████████████████████| 182 kB 72.7 MB/s 
     |████████████████████████████████| 132 kB 73.6 MB/s 
     |████████████████████████████████| 127 kB 74.0 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 5.0 MB/s 
     |████████████████████████████████| 7.6 MB 58.7 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simp

In [2]:
from huggingface_hub import notebook_login
notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.huggingface/token
Login successful


In [3]:
! apt install git-lfs

Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (2.3.4-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 11 not upgraded.


In [4]:
from datasets import load_dataset, load_metric, Audio

common_voice_train = load_dataset("common_voice", "tr", split="train+validation")
common_voice_test = load_dataset("common_voice", "tr", split="test") 

# common_voice -> dataset name
# tr is the language code corresponding to Turkish - configuration name

Generating train split:   0%|          | 0/1831 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1647 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1647 [00:00<?, ? examples/s]

Generating other split:   0%|          | 0/325 [00:00<?, ? examples/s]

Generating validated split:   0%|          | 0/18685 [00:00<?, ? examples/s]

Generating invalidated split:   0%|          | 0/1726 [00:00<?, ? examples/s]

Dataset common_voice downloaded and prepared to /root/.cache/huggingface/datasets/common_voice/tr/6.1.0/a1dc74461f6c839bfe1e8cf1262fd4cf24297e3fbd4087a711bd090779023a5e. Subsequent calls will reuse this data.


In [5]:
common_voice_train.description

"Common Voice is Mozilla's initiative to help teach machines how real people speak.\nThe dataset currently consists of 7,335 validated hours of speech in 60 languages, but we’re always adding more voices and languages.\n"

 > using the turkish language to finetune XLS_R model
 

In [6]:
common_voice_train = common_voice_train.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "segment", "up_votes"])
common_voice_test = common_voice_test.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "segment", "up_votes"])


In [7]:
# display random samples of the dataset

from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import HTML

def show_random_elements(dataset, num_examples=10):
  assert num_examples <= len(dataset)
  picks = []
  for _ in range(num_examples):
    pick = random.randint(0, len(dataset)-1)
    while pick in picks:
      pick = random.randint(0, len(dataset)-1)
    picks.append(pick)

  df = pd.DataFrame(dataset[picks])
  display(HTML(df.to_html()))

show_random_elements(common_voice_train.remove_columns(["path", "audio"]))

,sentence
0,Elimizde video kayıtları var.
1,"""Anlaşma Lefkoşa'da imzalandı."""
2,Şu anda özelleştirme sözleşmesini hazırlanıyor.
3,Anlaşmanın son imzalanma tarihiyse bir Temmuz.
4,Yeni anayasanın onaylamasına çalışılacak.
5,Kredilerin büyük bölümü özel şirketlere verildi.
6,Zaman içinde ikilinin arasında bir aşk doğuyor.
7,"Terör, fazla önem vermediğimiz bir sorun."
8,"""Bu toprakların özü İslam'la yoğrulmuştur."""
9,Demokratik partilerin iktidara geldiği iki bin yılından bu yana ilerleme oldu mu?


In [8]:
import re
chars_to_remove_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\?\']'

def remove_special_characters(batch):
  batch["sentence"] = re.sub(chars_to_remove_regex, '', batch["sentence"]).lower()
  return batch

show_random_elements(common_voice_train.remove_columns(["path", "audio"]))

,sentence
0,Ancak seçimlerden net bir galip çıkmadı.
1,Biz tam desteğimizi garanti ediyoruz.
2,Bayanlar sekizlide Rumenler son şampiyondular.
3,Fikir bir miktar direnişle de karşılaşmadı değil.
4,Yolsuzluk ve rüşvet yok kabul ediliyor.
5,Batı Balkanlar nereye doğru gidiyor?
6,Posavina Hırvatlarının sayısı azalıyor.
7,O yüzden de bazen keşke o olabilseydim diyorum.
8,En büyük ve en değerli firmalar hala devlet malı.
9,Ancak bu bilgiler kamuoyuna açıklanmayacak.


In [9]:
common_voice_train = common_voice_train.map(remove_special_characters)
common_voice_test = common_voice_test.map(remove_special_characters)

  0%|          | 0/3478 [00:00<?, ?ex/s]

  0%|          | 0/1647 [00:00<?, ?ex/s]

In [10]:
show_random_elements(common_voice_train.remove_columns(["path", "audio"]))

,sentence
0,bu turda en çok oyu alan aday kazanıyor
1,bugüne kadar varılan anlaşmaların uygulanabilirliği nedir
2,polis varlığı barizdi
3,sonunda taşınmak zorunda kaldılar
4,douka ödülü kazanan dokuzuncu kişi oldu
5,on iki tane de bağımsız aday var
6,bu tür yatırımlar aracılığıyla bütçeye ne oranda bir kaynak girişi sağlanması bekleniyor
7,asıl geri ödeme tarihi iki bin yirmi bir idi
8,dokunan desen oldukça zor ve karışık
9,asgari fiyat altı milyon avro olarak belirlendi


> remove the caps ^ from letters

In [11]:
def replace_hatted_characters(batch):
  batch["sentence"] = re.sub('[â]', 'a', batch["sentence"])
  batch["sentence"] = re.sub('[î]', 'i', batch["sentence"])
  batch["sentence"] = re.sub('[ô]', 'o', batch["sentence"])
  batch["sentence"] = re.sub('[û]', 'u', batch["sentence"])
  return batch

In [12]:
common_voice_train = common_voice_train.map(replace_hatted_characters)
common_voice_test = common_voice_test.map(replace_hatted_characters)

  0%|          | 0/3478 [00:00<?, ?ex/s]

  0%|          | 0/1647 [00:00<?, ?ex/s]

- CTC: classify speech chunks into letters
- Extract all distinct letters of the training and test data and build a vocabulry from the set of letters

In [13]:
def extract_all_chars(batch):
  all_text = " ".join(batch["sentence"])
  vocab = list(set(all_text))
  return {"vocab":[vocab], "all_text":[all_text]}

In [14]:
vocab_train = common_voice_train.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=common_voice_train.column_names)
vocab_test = common_voice_test.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=common_voice_test.column_names)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [15]:
vocab_list = list(set(vocab_train["vocab"][0]) | set(vocab_test["vocab"][0]))
vocab_dict = {v: k for k, v in enumerate(sorted(vocab_list))}
# vocab_dict

In [16]:
# vocab json file

import json

with open('turkish_vocab.json', 'w') as vocab_file:
  json.dump(vocab_dict, vocab_file)

In [54]:
# use the json file to load the vocabulary into the instance of wav2vec tokenizer class

from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer("./turkish_vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

In [61]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Thu Dec  8 14:15:57 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P0    28W /  70W |   2982MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [62]:
repo_name = "wa2vec2-large-xls-r-colab_turkish"

In [63]:
tokenizer.push_to_hub(repo_name)

tokenizer config file saved in wa2vec2-large-xls-r-colab_turkish/tokenizer_config.json
Special tokens file saved in wa2vec2-large-xls-r-colab_turkish/special_tokens_map.json
Uploading the following files to ix502iv/wa2vec2-large-xls-r-colab_turkish: tokenizer_config.json,vocab.json,special_tokens_map.json


CommitInfo(commit_url='https://huggingface.co/ix502iv/wa2vec2-large-xls-r-colab_turkish/commit/33f91a121004986471769001fd1ae0d25e62e910', commit_message='Upload tokenizer', commit_description='', oid='33f91a121004986471769001fd1ae0d25e62e910', pr_url=None, pr_revision=None, pr_num=None)

Feature Extraction

In [64]:
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0,
                                             do_normalize=True, return_attention_mask=True)

In [65]:
from transformers import Wav2Vec2Processor # combine model and processor object
processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

Preprocess Data

In [66]:
common_voice_train = common_voice_train.cast_column("audio", Audio(sampling_rate=16_000))
common_voice_test = common_voice_test.cast_column("audio", Audio(sampling_rate=16_000))


ValueError: ignored

In [67]:
import IPython.display as ipd
import numpy as np
import random

rand_int = random.randint(0, len(common_voice_train)-1)

print(common_voice_train[rand_int]["sentence"])
ipd.Audio(data=common_voice_train[rand_int]["audio"]["array"], autoplay=True,
          rate=16000)

KeyError: ignored

In [ ]:
rand_int = random.randint(0, len(common_voice_train)-1)

print("Target text:", common_voice_train[rand_int]["sentence"])
print("Input array shape:", common_voice_train[rand_int]["audio"]["array"].shape)
print("Sampling rate:", common_voice_train[rand_int]["audio"]["sampling_rate"])

In [ ]:
def prepare_dataset(batch):
    audio = batch["audio"]

    # batched output is "un-batched"
    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    batch["input_length"] = len(batch["input_values"])
    
    with processor.as_target_processor():
        batch["labels"] = processor(batch["sentence"]).input_ids
    return batch

In [ ]:
common_voice_train = common_voice_train.map(prepare_dataset, remove_columns=common_voice_train.column_names)
common_voice_test = common_voice_test.map(prepare_dataset, remove_columns=common_voice_test.column_names)

In [ ]:
! pip install torch

In [ ]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [ ]:
data_collator = DataCollatorCTCWithPadding(processor=processor,padding=True )

In [ ]:
wer_metric = load_metric("wer")

In [ ]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [ ]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-xls-r-300m", 
    attention_dropout=0.0,
    hidden_dropout=0.0,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.0,
    ctc_loss_reduction="mean", 
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer),
)

In [ ]:
model.freeze_feature_extractor()

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir=repo_name,
  group_by_length=True,
  per_device_train_batch_size=16,
  gradient_accumulation_steps=2,
  evaluation_strategy="steps",
  num_train_epochs=30,
  gradient_checkpointing=True,
  fp16=True,
  save_steps=400,
  eval_steps=400,
  logging_steps=400,
  learning_rate=3e-4,
  warmup_steps=500,
  save_total_limit=2,
  push_to_hub=True,
)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=common_voice_train,
    eval_dataset=common_voice_test,
    tokenizer=processor.feature_extractor,
)

In [ ]:
trainer.train()